In [49]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

from scipy.stats import entropy


import khaterehlib



### make sure you import with correct file name above

```
import filename as lib
```
(without the .py extension)

if filename is lib.py
```
import lib as lib
```

if filename is res.py
```
import res as lib
```

In [50]:

# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [51]:
pk = np.array([1/5, 2/5, 2/5])  # fair coin
H = entropy(pk)
print(H)

1.0549201679861442


In [52]:
import khaterehlib

list_labels = [1, 2, 0, 1, 2, 0]
uniq_labels = [0, 1, 2]
probs = khaterehlib.calculate_probabilities(list_labels, uniq_labels)

print(probs)

{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}


In [53]:


import khaterehlib
# test your function
list_probas=[1/5, 2/5, 2/5]
print(khaterehlib.calc_entropy_from_probabilities(list_probas))
# above should print 1.054...

1.5219280948873621


In [54]:



#test your function
old_entropy=1
new_entropies=[0,0.65]
count_items=[4,6]
print(khaterehlib.information_gain(old_entropy,new_entropies,count_items))
# above should print 0.61
    
    
    

0.61


In [55]:
num_feats=X_train.shape[1]
print(khaterehlib.initialize_weights(num_feats))

[2 2 2 2]


### Task4: PSO Implementation

#### Modified the entropy function to get a vector of entropies for n particles

In [56]:
### Below we just randomly assign 100 particles and see if we can find the global minimum.
### THis is just to check

In [57]:
params1=[khaterehlib.initialize_weights(X_train.shape[1]) for i in range(100)]# a vector of shape 100,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(100)]# a vector of shape 100
# use the np.random.uniform() function

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1=np.array(params1)
params2=np.array(params2)

print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 4)
Shape of params 2 (thresholds) (100,)


In [58]:

z = khaterehlib.objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum
param1_min = params1[z.argmin()] # use z.argmin()
param2_min = params2[z.argmin()] # use z.argmin()

print("param1_min",param1_min,"param2_min",param2_min)


param1_min [2 2 2 2] param2_min 0.9183174844620484


In [59]:
### Setting up the particles and other parameters now

In [60]:

# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1=[khaterehlib.initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 4) params2 shape is  (20,)


In [61]:
# define velocity of each weight of every particle
V_param1 = [khaterehlib.initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = khaterehlib.objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [1.58306891 1.58306891 1.58306891 1.1863708  1.58306891 1.58306891
 1.1863708  1.1863708  1.58306891 1.58306891 1.1863708  1.58306891
 1.58306891 1.58306891 1.58306891 1.18516434 1.1863708  1.58306891
 1.1863708  1.58306891]
gbest obj value among all 20 particles is as follows 1.185164339303613


### the update function

In [62]:
def update():
    "Function to do one iteration of particle swarm optimization"
    global V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = khaterehlib.objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj
 


In [63]:
for i in range(100):
    update()
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min],khaterehlib.objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([ 2.00006965e+00, -4.97631552e-04,  2.00000000e+00,  2.00006798e+00]), 1.518392392388451))=0.6665656552860705
Global optimal at f([array([2, 2, 2, 2]), 0.9183174844620484])=1.185164339303613


### Try to resolve the error above

### Try a different dataset (preferable binary classification)

In [64]:
# import some data to play with
#load the breast cancer dataset 
from sklearn import datasets
bcan = datasets.load_breast_cancer()
X = bcan.data
y = bcan.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
print("Shapr of training data is ",X_train.shape)

Shapr of training data is  (381, 30)


In [65]:
# initialize the params1 and params2
n_params1 = 100 # number of weight vectors
n_params2 = 100 # number of thresholds

# call the initialize_weights function above
params1 = khaterehlib.initialize_weights(n_params1)

# use the np.random.uniform() function
params2 = np.random.uniform(0, 1, n_params2)

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1 = np.array(params1)
params2 = np.array(params2)

print("Shape of params 1 (weights)", params1.shape)
print("Shape of params 2 (thresholds)", params2.shape)


Shape of params 1 (weights) (100,)
Shape of params 2 (thresholds) (100,)


In [66]:
z = lib.objective_fn_vector(params1, params2, X_train, y_train)

# Find the global minimum
idx_min = z.argmin()
param1_min = params1[idx_min]
param2_min = params2[idx_min]

print("param1_min",param1_min,"param2_min",param2_min)


param1_min 2 param2_min 0.459553670010184


In [67]:
# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1=[lib.initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)


params1 shape is  (20, 30) params2 shape is  (20,)


In [68]:
# define velocity of each weight of every particle
V_param1 = [khaterehlib.initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = khaterehlib.objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.95844882 0.95844882 0.95844882 0.95844882 0.95844882 0.95844882
 0.95844882 0.95844882 0.95844882 0.95844882 0.95844882 0.95844882
 0.95844882 0.95844882 0.95844882 0.95844882 0.95844882 0.95844882
 0.95844882 0.95844882]
gbest obj value among all 20 particles is as follows 0.9584488208225608


In [69]:
def update():
    "Function to do one iteration of particle swarm optimization"
    global V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = khaterehlib.objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj

In [70]:
for i in range(1000):
    update()
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], lib.objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([ 2.00000000e+00,  2.00000000e+00,  9.27029896e-82,  2.00000000e+00,
        2.00000000e+00,  2.00000000e+00,  1.98351716e-45,  1.76950001e-46,
        2.00000000e+00,  2.54685300e-01,  2.00000000e+00,  5.17308985e-01,
        2.00000000e+00,  2.00000000e+00, -1.07516696e-72, -1.52565466e-07,
       -2.31613849e-71,  8.60504297e-81,  2.00000000e+00,  2.00000000e+00,
        2.00000000e+00,  2.00000000e+00,  6.40166550e-05,  2.00000000e+00,
        2.00000000e+00,  2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
        2.00000000e+00, -2.04858417e-43]), 0.7160866152562138))=0.9584488208225608
Global optimal at f([2, 0.459553670010184])=0.9584488208225608
